In [ ]:
%cd /home/adityasidharta/git/shopee_data_science
%env PROJECT_PATH /home/adityasidharta/git/shopee_data_science

In [ ]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
from utils.envs import *
import string
from sklearn.model_selection import train_test_split
from utils.common import create_directory
from pathlib import Path
from category_encoders import OrdinalEncoder

In [ ]:
create_directory(os.path.join(result_path, 'static'))
RESULT_PATH = Path(os.path.join(result_path, 'static'))

In [ ]:
raw_beauty_train = pd.read_csv(beauty_train_repo)[['title']]
raw_beauty_val = pd.read_csv(beauty_test_repo)[['title']]

raw_fashion_train = pd.read_csv(fashion_train_repo)[['title']]
raw_fashion_val = pd.read_csv(fashion_test_repo)[['title']]

raw_mobile_train = pd.read_csv(mobile_train_repo)[['title']]
raw_mobile_val = pd.read_csv(mobile_test_repo)[['title']]

In [ ]:
full_text_df = pd.concat([raw_beauty_train, raw_beauty_val, raw_fashion_train, raw_fashion_val, raw_mobile_train, raw_mobile_val])

In [ ]:
full_text_df.columns = ['text']
full_text_df['label'] = 0
full_text_df = full_text_df[['label', 'text']]

In [ ]:
train_df, val_df = train_test_split(full_text_df, test_size = 0.10)

In [ ]:
data_lm = TextLMDataBunch.from_df(path='',train_df=train_df, valid_df=val_df)
data_lm.show_batch()

In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn_lm.lr_find()

In [ ]:
learn_lm.recorder.plot()

In [ ]:
learn_lm.fit_one_cycle(1, 0.1, moms=(0.8,0.7))

In [ ]:
TEXT = "blus wanita casual"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
data_lm.save(RESULT_PATH/'data_lm.pkl')
learn_lm.save(RESULT_PATH/'model_lm.pkl')
learn_lm.save_encoder(RESULT_PATH/'encoder_lm.pkl')